In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.models import Sequential
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from PIL import Image

In [2]:
! pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import opendatasets as od


In [4]:
data_url = 'https://www.kaggle.com/competitions/paddy-disease-classification/overview'


In [5]:
od.download(data_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aswinramanathan
Your Kaggle Key: ··········


100%|██████████| 1.02G/1.02G [00:04<00:00, 267MB/s]



Extracting archive ./paddy-disease-classification/paddy-disease-classification.zip to ./paddy-disease-classification


In [6]:
data_dir='/content/paddy-disease-classification'

In [7]:
!ls -lh {data_dir}

total 492K
-rw-r--r--  1 root root  45K Jul 27 09:42 sample_submission.csv
drwxr-xr-x  2 root root 112K Jul 27 09:42 test_images
-rw-r--r--  1 root root 328K Jul 27 09:42 train.csv
drwxr-xr-x 12 root root 4.0K Jul 27 09:42 train_images


In [8]:
!ls -lh {data_dir}/test_images

total 268M
-rw-r--r-- 1 root root  92K Jul 27 09:42 200001.jpg
-rw-r--r-- 1 root root 100K Jul 27 09:42 200002.jpg
-rw-r--r-- 1 root root  80K Jul 27 09:42 200003.jpg
-rw-r--r-- 1 root root  89K Jul 27 09:42 200004.jpg
-rw-r--r-- 1 root root  45K Jul 27 09:42 200005.jpg
-rw-r--r-- 1 root root  90K Jul 27 09:42 200006.jpg
-rw-r--r-- 1 root root  90K Jul 27 09:42 200007.jpg
-rw-r--r-- 1 root root  60K Jul 27 09:42 200008.jpg
-rw-r--r-- 1 root root  85K Jul 27 09:42 200009.jpg
-rw-r--r-- 1 root root  93K Jul 27 09:42 200010.jpg
-rw-r--r-- 1 root root  59K Jul 27 09:42 200011.jpg
-rw-r--r-- 1 root root  79K Jul 27 09:42 200012.jpg
-rw-r--r-- 1 root root  88K Jul 27 09:42 200013.jpg
-rw-r--r-- 1 root root  73K Jul 27 09:42 200014.jpg
-rw-r--r-- 1 root root  91K Jul 27 09:42 200015.jpg
-rw-r--r-- 1 root root  93K Jul 27 09:42 200016.jpg
-rw-r--r-- 1 root root  80K Jul 27 09:42 200017.jpg
-rw-r--r-- 1 root root  80K Jul 27 09:42 200018.jpg
-rw-r--r-- 1 root root  93K Jul 27 09:42 200019.jpg
-

# First Submission


In [ ]:
#hello vanakam
train_data_dir = '/content/paddy-disease-classification/train_images/'
img_height = 225
img_width = 225
batch_size = 256

train_datagen = ImageDataGenerator(rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 8330 images belonging to 10 classes.
Found 2077 images belonging to 10 classes.


In [ ]:
train_generator.class_indices

{'bacterial_leaf_blight': 0,
 'bacterial_leaf_streak': 1,
 'bacterial_panicle_blight': 2,
 'blast': 3,
 'brown_spot': 4,
 'dead_heart': 5,
 'downy_mildew': 6,
 'hispa': 7,
 'normal': 8,
 'tungro': 9}

In [ ]:
folder_path = '/content/paddy-disease-classification/train_images/'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

CNN model

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation="relu",input_shape=(225,225,3)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Conv2D(32,(5,5),activation="relu",input_shape=(225,225,5)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Conv2D(64,(5,5),activation="relu",input_shape=(225,225,5)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Conv2D(64,(5,5),activation="relu",input_shape=(225,225,5)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Flatten(),
                                    ##
                                    tf.keras.layers.Dense(512,activation="relu"),
                                    ##
                                    tf.keras.layers.Dense(10,activation="softmax")
])

In [ ]:
model1 = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation="relu",input_shape=(225,225,3)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Conv2D(32,(5,5),activation="relu"),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Conv2D(64,(5,5),activation="relu"),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Conv2D(64,(5,5),activation="relu"),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Conv2D(128,(5,5),activation="relu"),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Conv2D(256,(5,5),activation="relu"),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    ##
                                    tf.keras.layers.Flatten(),
                                    ##
                                    tf.keras.layers.Dense(1024,activation="relu"),
                                    ##
                                    tf.keras.layers.Dense(10,activation="softmax")
])

# IDEAS TO IMPROVE MODEL ACCURACY AND COMPUTATION SPEED
# 1) Grayscale images 
# 2) Reduce batch size
# 3) explore other optimizers 
# 4) reduce training set size
# 5) use seed in flow from directory
# 6) visualize how each layer works !!important!!
# 7) evaluate model
# 8) multilabel classification possibility

In [ ]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
nb_epochs = 15
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/15
32/32 [==============================] - 77s 2s/step - loss: 2.1833 - accuracy: 0.1875 - val_loss: 2.1016 - val_accuracy: 0.2305
Epoch 2/15
32/32 [==============================] - 67s 2s/step - loss: 2.0194 - accuracy: 0.2601 - val_loss: 1.9137 - val_accuracy: 0.3394
Epoch 3/15
32/32 [==============================] - 71s 2s/step - loss: 1.8083 - accuracy: 0.3620 - val_loss: 1.7523 - val_accuracy: 0.3682
Epoch 4/15
32/32 [==============================] - 62s 2s/step - loss: 1.6949 - accuracy: 0.3950 - val_loss: 1.6508 - val_accuracy: 0.4009
Epoch 5/15
32/32 [==============================] - 72s 2s/step - loss: 1.5684 - accuracy: 0.4477 - val_loss: 1.4997 - val_accuracy: 0.4834
Epoch 6/15
32/32 [==============================] - 64s 2s/step - loss: 1.4044 - accuracy: 0.5130 - val_loss: 1.4252 - val_accuracy: 0.5088
Epoch 7/15
32/32 [==============================] - 65s 2s/step - loss: 1.2664 - accuracy: 0.5690 - val_loss: 1.2911 - val_accuracy: 0.5635
Epoch 8/15
32/32 [==

In [ ]:
! ls /content/paddy-disease-classification/train_images 

bacterial_leaf_blight	  blast       downy_mildew  tungro
bacterial_leaf_streak	  brown_spot  hispa
bacterial_panicle_blight  dead_heart  normal


In [ ]:
dictolist1 = ['bacterial_leaf_blight','bacterial_leaf_streak',
 'bacterial_panicle_blight',
 'blast',
 'brown_spot',
 'dead_heart',
 'downy_mildew',
 'hispa',
 'normal',
 'tungro']


In [ ]:
import csv

In [ ]:
%%time

dir_path = "/content/paddy-disease-classification/test_images"
fields = ['image_id','label']
filename = '/content/paddy-disease-classification/samplesubmissiontest.csv'
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
    for i in os.listdir(dir_path):
      img = image.load_img(dir_path+'//'+i,target_size=(225,225))
    
      
      x = image.img_to_array(img)
      x = np.expand_dims(x,axis=0)
      images = np.vstack([x])
      val = model.predict(images)
      index = np.argmax(val, axis=-1)
      row = [i,dictolist1[index[0]]]
      csvwriter.writerow(row)

CPU times: user 2min 44s, sys: 6.91 s, total: 2min 51s
Wall time: 2min 48s


# Second Submission

In [ ]:
train_data_dir = '/content/paddy-disease-classification/train_images/'
img_height = 225
img_width = 225
batch_size = 256

train_datagen = ImageDataGenerator(rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')